```markdown
To run this Jupyter notebook, use Python 3.13 or later. The recommended environment is a virtual environment or a conda environment with the necessary packages installed, as specified int he requirements.txt file. Ensure that you have Jupyter Notebook installed to execute the cells interactively.
```

In [ ]:
%run reuse.ipynb

In [58]:
import ipywidgets as widgets
from IPython.display import display

# Create a dropdown widget for environment selection
env_dropdown = widgets.Dropdown(
    options=['dev', 'uat', 'prod'],
    value='dev',
    description='Environment:',
)

outputdirectory = widgets.Text("testdata")

# Display the widget
display(widgets.HBox([env_dropdown, outputdirectory]))




In [60]:
selected_env = env_dropdown.value
outputdir = outputdirectory.value
healthData = HealthData(selected_env,outputdir) # type: ignore
healthData.generate_data()

In [ ]:
import pandas as pd

healthdatadf = pd.read_parquet(f"./{outputdir}/health_data_{selected_env}.parquet")

In [ ]:
####!pyspark --packages io.delta:delta-core_2.11:0.4.0
import pyspark
from pyspark.sql import SparkSession
from delta import *


builder = (
    SparkSession.builder.appName("DeltaTableCreation")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)


In [ ]:
filtered_healthdatadf = healthdatadf[healthdatadf['BloodPressure'] > 120]
print(filtered_healthdatadf)

# Use Spark SQL to filter the data and convert to Pandas DataFrame
spark.createDataFrame(healthdatadf).createOrReplaceTempView("sparkhealthdatadf")
filtered_healthdatadf_sql = spark.sql("SELECT * FROM sparkhealthdatadf WHERE BloodPressure > 120").toPandas()
print(filtered_healthdatadf_sql)

In [ ]:
# Create a Spark DataFrame from the healthdatadf DataFrame
sparkdf = spark.createDataFrame(healthdatadf)

# Write the DataFrame as a Delta table
sparkdf.write.format("delta").mode("overwrite").save(f"./{outputdir}/health_data_{selected_env}")

In [ ]:
# Read the Delta table
delta_table = spark.read.format("delta").load(f"./{outputdir}/health_data_{selected_env}")

filtered_data = delta_table.filter(delta_table.BloodPressure > 120)
filtered_data.show()

In [ ]:
# Create a temporary view
delta_table.createOrReplaceTempView(f"health_data")

# Use Spark SQL to filter the data
filtered_data_sql = spark.sql("SELECT * FROM health_data WHERE BloodPressure > 120")
filtered_data_sql.show()